# Concluding the Exoplanet Composite Discovery Method Predictor

## Preprocessing

### In the preprocessing ipynb, I started by using pandas to read the raw csv, made a copy and displayed the raw csv info
### Using a missing value threshold of 0, I removed columns that exceeded this threshold. 0 made the most sense as a threshold to simplify model training and reduce problem complexity. 
### After reducing the number of features, I used the exoplanet archive column mapping csv to map the table name (ex: pl_name) to the Description (ex: Planet name) using a dictionary
### Used a for loop through the number of code columns (358) and their definitions, where if row i in the database column matches a column name in the filtered raw data, you assign the key and value to the dictionary, removing trailing and leading whitespaces
### Used boolean to confirm we mapped all database column names to their definitions
### Then just reassigned the raw data column values (the names) as the values of the dictionary {key:value}
### Checkpoint where we create a copy of the current dataset we are working with
### Then I checked unique values in the "Discovery Method" column in the dataset to get an idea of the discovery methods possible
### Using value_count, I got an idea of the frequency of each of the discovery methods in the column. Transit was by far the most frequent
### Some of the discovery methods only had 10 instances, so I used SMOTE (synthetic minority oversampling technique) to synthetically generate instances of the minority class for a better class balance. This is essential for machine learning models to properly learn how to predict the discovery methods and not just predict the most frequent discovery method. 
### Realized that if I encoded Discovery Facility, Discovery Telescope and Discovery Instrument, there are 70-90 unique values for EACH, so encoding these was determined unfeasable
### So I dropped them along with Planet name and Host name, which would be irrelevant to model training
### Then I removed all the one hot binary encoded variables "Detected by..." columns except for transits
### This is because if "Detected by Transits" is 0, we already know that it has to be one of the other 10 discovery methods
### Before doing that, I removed instances of exoplanets that were found by more than one discovery method for more straightforward model training
### This was done by summmating each of the target dummies for each of the exoplanets, and if the sum was greater than 1 that means it was discovered by more than one method, warranting its removal
### After checking the new observation count for each of the "Detected by..." methods, we dropped the unnecessary "Detected by..." columns
### Checking the ratio of "Detected by transits" to the total number of instances, I saw it transits was roughly 71% of discovery method instances
### Imported SMOTE and specified my features (all variables except "Detected by Transits") and targets ("Detected by Transits")
### The new ratio was 50%, implying SMOTE worked as intended (50% exoplanets discovered by Transits, 50% were not)
### Then proceeded with standardization; Removed columns that could not be standardized (Discovery Year, Circumbinary Flag, Controversial Flag) and added these after standardizing
### Imported StandardScaler function from sklearn preprocessing library and fit the scaler to all the unscaled features.
### Then applied scaler.transform to all the unscaled features besides the ones excluded above. Checked the shapes of features, targets and the variables we removed, they all must match
### Since we used StandardScaler the scaled features are now in numpy array format. To combine it with the other excluded variables, it must first be convered to a pandas dataframe. Then we just added the excluded variables to the new pandas dataframe of the scaled features
### Converted discovery year to be treated as a categorical variable for the purposes of this project
### One last checkpoint (creating copy of the current dataframe we are working with) 
### Finally export the preprocessed dataframe and turn it into a csv to be saved in the current directory